In [133]:
#Goal: given a sentence, get the right topic
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import trigrams
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.stem import wordnet, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.probability import FreqDist
import pandas as pd 
import numpy as np 

In [248]:
samplestring = "My stomach hurts "

In [249]:
samplestring_tokens = word_tokenize(samplestring)
samplestring_tokens

['My', 'stomach', 'hurts']

In [250]:
#Filter unnecessary stopwords
stopWords = set(stopwords.words('english'))
samplestring_tokens_filtered = []

for w in samplestring_tokens:
    if w.lower() not in stopWords:
        samplestring_tokens_filtered.append(w)

samplestring_tokens_filtered

['stomach', 'hurts']

In [251]:
#We won't need punctuations either (we would need numbers however)
samplestring_tokens_filtered_postpunctuation = []
punctuation = re.compile(r'[-.?!,:;()|]')
for w in samplestring_tokens_filtered:
    word = punctuation.sub("",w)
    if len(word) > 0:
        samplestring_tokens_filtered_postpunctuation.append(word)
        
samplestring_tokens_filtered_postpunctuation

['stomach', 'hurts']

In [252]:
#If need to token per every 3 words (then analyze every 3 words for context -- that or for further data analysis
samplestring_trigrams = list(nltk.trigrams(samplestring_tokens_filtered))
samplestring_trigrams

[]

In [253]:
#List of symtoms all sorted for binary search
symptoms = pd.read_csv('diagnosis_analysis/sym_t.csv')
symptoms = symptoms.dropna()
symstr = ' '.join(symptoms["symptom"])
# symptoms = symptoms["symptom"].sort_values(ascending=True)
# symptoms = symptoms.dropna()
# symptoms
symstr

"Upper abdominal pain Lower abdominal pain Abscess (Collection of pus) Alcohol abuse Anxiety (Nervousness) Arm ache or pain Back ache or pain Bleeding tendency Blood in vomit Bloody diarrhea Pain or soreness of breast Calf pain Chest pressure Chills Change in behavior Constipation Cough Dark stools Depressed Diarrhea Dizziness Double vision (Diplopia) Ear pressure Pain in the ear Elbow ache or pain Eye pain (Irritation) Facial pain Fainting Fever Fever in the returning traveler Fever of unknown origin Flank pain Frequent urination (Frequency) Foot pain Cranky, crying, fussy, irritable child Groin pain Delusions or hallucinations Hand, finger ache or pain Head injury Headache Heel pain Heat illness Hip pain Hives Hoarse voice Hypothermia (Low temperature) Incontinence (leaking urine) Insect sting Insomnia (Trouble sleeping) Skin itching Joint pain Kidney pain (Flank pain) Knee pain Laceration Leg ache or pain Swelling of both legs Lethargy (Sluggishness) Mouth pain Muscle pain Nail Inju

In [254]:
#List of key words, sorted
rawsymtoken = nltk.word_tokenize(symstr)
symptokens = [x.lower() for x in rawsymtoken]
symptokens
symfdist = FreqDist(symptokens)
symvocab = sorted(symfdist.keys())
symoccurences = {}
#hardcoded whee
for key in symvocab:
    symoccurences[key] = []
    for s in symptoms["symptom"]:
        if key in s.lower():
            symoccurences[key].append(s)
#symvocab
symoccurences

{"'s": ["Traveler's diarrhea"],
 '(': ['Abscess (Collection of pus)',
  'Anxiety (Nervousness)',
  'Double vision (Diplopia)',
  'Eye pain (Irritation)',
  'Frequent urination (Frequency)',
  'Hypothermia (Low temperature)',
  'Incontinence (leaking urine)',
  'Insomnia (Trouble sleeping)',
  'Kidney pain (Flank pain)',
  'Lethargy (Sluggishness)',
  'Substance abuse (Drug abuse)',
  'Swallowing problem (Dysphagia)',
  'Unsteady gait (Trouble walking)',
  'Vertigo (Room spinning)',
  'Weakness (Muscle localized)',
  'Wheezing (Noisy breathing)',
  'Swollen lymph nodes (Large lymph nodes)',
  'Itchy rash (Pruritic rash)',
  'Blood in urine (Hematuria)',
  'Urinary retention (Inability to urinate)',
  'Liver failure (Cirrhosis)',
  'Melena (Black stools from blood)',
  'Ringing in ears (Tinnitus)',
  'Cyanosis (Blue skin coloration)',
  'Abdominal swelling (Stomach swelling)',
  'Hand numbness (paresthesias)',
  'Hemoptysis (Coughing blood)',
  'Jaundice (Yellowing skin)',
  'Flatulence 

In [255]:
def binary_search(item_list,item):
    first = 0
    last = len(item_list)-1
    found = False
    while( first<=last and not found):
        mid = (first + last)//2
        #print(item)
        #print("YO U MID",mid)
        #print(type(item_list[mid]))
        if item_list[mid] == item :
            found = True
        else:
            if item < item_list[mid]:
                last = mid - 1
            else:
                first = mid + 1	
    return found

In [257]:
SYMPTOMS_LIST = []

#Let's filter the verbs before we lemma and stem 
#Not sure if we need to chunk. Depends on possible user language symptoms
word_lem = WordNetLemmatizer()
samplestring_verbs = []
for tkn in samplestring_tokens_filtered_postpunctuation:
    synonyms = []
    POStype = nltk.pos_tag([tkn])
    print(POStype)
    result = binary_search(symvocab,POStype[0][0].lower())
    if result == True:
        print(POStype[0][0].upper(),"EXISTS1") #the word user type exists as it is in list of symptoms
        print(symoccurences[POStype[0][0].lower()])
        SYMPTOMS_LIST.append(symoccurences[POStype[0][0].lower()])
        #no need to go and find all these synonyms as we can go find the symptoms
    syns1 = wordnet.synsets(POStype[0][0])
    print("From the original",syns1)
    for x in syns1:
        #this loop only looks through the 1st level synonym
        syns1type = re.search('^(.*?)\..*',x.name()).group(0)
        if not (syns1type == "n" and POStype[0][1][0].lower() == "v") or (syns1type == "v" and POStype[0][1][0].lower() == "n"):
            #at least no unnecessary search bet noun and verb
            keyword = x.name().lower().split(".")[0]
            print("KEYWORD",keyword)
            if keyword != POStype[0][0].lower() and keyword not in synonyms:
                synonyms.append(keyword)
                print("SEARCHING FOR...",keyword)
                result = binary_search(symvocab,keyword)
                if result == True:
                    print(keyword.upper(),"EXISTS2") #a direct synonym of the user word exist in the list
                    print(symoccurences[keyword])
                    SYMPTOMS_LIST.append(symoccurences[keyword])
            #let's look at the other nyms
            others = []
            hypernyms = [others.append(y.name().lower().split(".")[0]) for y in x.hypernyms()]
            for i,j in enumerate(x.hypernyms()):
                #print("HYPONYM",x.hypernyms()[i].hyponyms())
                for z in x.hypernyms()[i].hyponyms():
                    others.append(z.name().lower().split(".")[0])
            lemmas = [others.append(l.lower()) for l in x.lemma_names()]
            print("OTHERS",others)
            for word in others:
                if word not in synonyms:
                    synonyms.append(word)
                    print("SEARCHING FOR...",word)
                    result = binary_search(symvocab,word)
                    if result == True:
                        print(word,"EXISTS3")
                        print(symoccurences[word])
                        SYMPTOMS_LIST.append(symoccurences[word])
                
SYMPTOMS_LIST        

[('stomach', 'NN')]
STOMACH EXISTS1
['Abdominal swelling (Stomach swelling)', 'Stomach and abdominal pain']
From the original [Synset('stomach.n.01'), Synset('abdomen.n.01'), Synset('stomach.n.03'), Synset('stomach.n.04'), Synset('stomach.v.01'), Synset('digest.v.03')]
KEYWORD stomach
OTHERS ['internal_organ', 'excretory_organ', 'heart', 'hindgut', 'intestine', 'liver', 'respiratory_organ', 'stomach', 'viscera', 'stomach', 'tummy', 'tum', 'breadbasket']
SEARCHING FOR... internal_organ
SEARCHING FOR... excretory_organ
SEARCHING FOR... heart
heart EXISTS3
['Heart pulsations and palpitations', 'Heart murmur (Abnormal heart sound)', 'Low heart rate']
SEARCHING FOR... hindgut
SEARCHING FOR... intestine
SEARCHING FOR... liver
liver EXISTS3
['Liver failure (Cirrhosis)']
SEARCHING FOR... respiratory_organ
SEARCHING FOR... stomach
stomach EXISTS3
['Abdominal swelling (Stomach swelling)', 'Stomach and abdominal pain']
SEARCHING FOR... viscera
SEARCHING FOR... tummy
SEARCHING FOR... tum
SEARCHING

OTHERS ['be', 'abound', 'accept', 'account', 'account_for', 'act', 'answer', 'appear', 'bake', 'balance', 'be_well', 'beat', 'begin', 'begin', 'belong', 'belong', 'belong', 'belong', 'breathe', 'buy', 'clean', 'cohere', 'come_in_for', 'come_in_handy', 'compact', 'compare', 'confuse', 'connect', 'consist', 'consist', 'contain', 'contain', 'continue', 'cost', 'count', 'count', 'cover', 'cut', 'cut_across', 'deck', 'depend', 'deserve', 'disagree', 'distribute', 'diverge', 'draw', 'end', 'fall', 'fall', 'feel', 'figure', 'fit', 'gape', 'go', 'gravitate', 'hail', 'hang', 'head', 'hold', 'hoodoo', 'hum', 'impend', 'incarnate', 'iridesce', 'jumble', 'kill', 'lend', 'let_go', 'lie', 'litter', 'loiter', 'look', 'look', 'lubricate', 'make', 'make_sense', 'measure', 'mope', 'object', 'osculate', 'owe', 'pay', 'point', 'press', 'promise', 'prove', 'put_out', 'rage', 'range', 'rank', 'rate', 'recognize', 'relate', 'remain', 'represent', 'rest', 'retard', 'run', 'run_into', 'rut', 'seem', 'seethe', 

[['Abdominal swelling (Stomach swelling)', 'Stomach and abdominal pain'],
 ['Heart pulsations and palpitations',
  'Heart murmur (Abnormal heart sound)',
  'Low heart rate'],
 ['Liver failure (Cirrhosis)'],
 ['Abdominal swelling (Stomach swelling)', 'Stomach and abdominal pain'],
 ['Back ache or pain', 'Low back ache or pain'],
 ['Flank pain', 'Kidney pain (Flank pain)'],
 ['Hip pain'],
 ['Joint pain', 'Joint stiffness'],
 ['Visual flashing lights', 'Shin pain'],
 ['Shoulder ache or pain', 'Shoulder stiffness or tightness'],
 ['Toe pain'],
 ['Excessive appetite'],
 ['Hurts to breathe'],
 ['Head injury', 'Nail Injury', 'Nasal injury'],
 ['Heat illness'],
 ['Skin infection'],
 ['Skin trauma', 'Trauma', 'Headache after trauma'],
 ['Upper abdominal pain',
  'Lower abdominal pain',
  'Arm ache or pain',
  'Back ache or pain',
  'Pain or soreness of breast',
  'Calf pain',
  'Pain in the ear',
  'Elbow ache or pain',
  'Eye pain (Irritation)',
  'Facial pain',
  'Flank pain',
  'Foot pain',


In [205]:
def binary_search(item_list,item):
    first = 0
    last = len(item_list)-1
    found = False
    while( first<=last and not found):
        mid = (first + last)//2
        print(item)
        print("YO U MID",mid)
        print(type(item_list[mid]))
        if item_list[mid] == item :
            found = True
        else:
            if item < item_list[mid]:
                last = mid - 1
            else:
                first = mid + 1	
    return found

#For each of the keywords
for keyword in samplestring_tokens_filtered_postpunctuation:
    syns = wordnet.synsets(keyword)
    print(syns)
    for x in syns: 
        print(x)
        #result = binary_search(symvocab,x)
        #print("RESULT?",result)
        print("HYPERNIM",x.hypernyms())
        print("HYPERNIM",x.hypernyms()[0].hyponyms())
        print(x.lemma_names())
        print("\n")
    
    
'''
first we filter base on N or V
then for each of the synonym, we binary search the symptom list
(symptom list has to be alphabetically reordered) - until it finds something
'''

[Synset('nose.n.01'), Synset('nose.n.02'), Synset('nose.n.03'), Synset('nose.n.04'), Synset('nose.n.05'), Synset('nose.n.06'), Synset('nose.n.07'), Synset('nozzle.n.01'), Synset('intrude.v.03'), Synset('nose.v.02'), Synset('scent.v.02'), Synset('nose.v.04'), Synset('nuzzle.v.02'), Synset('nose.v.06')]
Synset('nose.n.01')
HYPERNIM [Synset('chemoreceptor.n.01')]
HYPERNIM [Synset('carotid_body.n.01'), Synset('nose.n.01'), Synset('tastebud.n.01')]
['nose', 'olfactory_organ']


Synset('nose.n.02')
HYPERNIM [Synset('front.n.04')]
HYPERNIM [Synset('bow.n.03'), Synset('facade.n.01'), Synset('face.n.08'), Synset('frontispiece.n.02'), Synset('nose.n.02'), Synset('nose.n.03'), Synset('nose_cone.n.01'), Synset('shirtfront.n.01'), Synset('shopfront.n.01')]
['nose']


Synset('nose.n.03')
HYPERNIM [Synset('front.n.04')]
HYPERNIM [Synset('bow.n.03'), Synset('facade.n.01'), Synset('face.n.08'), Synset('frontispiece.n.02'), Synset('nose.n.02'), Synset('nose.n.03'), Synset('nose_cone.n.01'), Synset('shir

IndexError: list index out of range

In [69]:
from nltk.corpus import wordnet
syns = wordnet.synsets("stomach")

In [70]:
syns

[Synset('stomach.n.01'),
 Synset('abdomen.n.01'),
 Synset('stomach.n.03'),
 Synset('stomach.n.04'),
 Synset('stomach.v.01'),
 Synset('digest.v.03')]

In [79]:
wordnet.synsets("hurting")
wordnet.synsets("hurting")[2].lemmas()[0]
#wordnet.synsets("hurting")[2].lemmas()[0].derivationally_related_forms()

Lemma('hurt.v.02.hurt')

In [72]:
wordnet.synsets("eye")

[Synset('eye.n.01'),
 Synset('eye.n.02'),
 Synset('eye.n.03'),
 Synset('center.n.01'),
 Synset('eye.n.05'),
 Synset('eye.v.01')]

In [47]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 

In [48]:
data = pd.read_excel('./Online Retail.xlsx') 
data.head() 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [49]:
data['Description'] = data['Description'].str.strip() #space in description strip
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str') #screw no invoice
data = data[~data['InvoiceNo'].str.contains('C')]  #screw credit
data.head() 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [66]:
basket_France = (data[data['Country'] =="United Kingdom"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [67]:
basket_France.head()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(basket_France["KNITTED UNION FLAG HOT WATER BOTTLE"])

InvoiceNo
536365       6.0
536366       0.0
536367       0.0
536368       0.0
536369       0.0
536371       0.0
536372       0.0
536373       6.0
536374       0.0
536375       6.0
536376       0.0
536377       0.0
536378       0.0
536380       0.0
536381       0.0
536382       0.0
536384       0.0
536385       0.0
536386       0.0
536387       0.0
536388       0.0
536390      24.0
536392       0.0
536393       0.0
536394       0.0
536395       0.0
536396       6.0
536397       0.0
536398       0.0
536399       0.0
536400       0.0
536401       0.0
536402       0.0
536404       0.0
536405       0.0
536406       6.0
536407       0.0
536408       0.0
536409       0.0
536412       0.0
536415       0.0
536416       0.0
536420       0.0
536423       0.0
536425       0.0
536437       0.0
536446       0.0
536460       0.0
536463       0.0
536464       0.0
536466       0.0
536477       0.0
536488       0.0
536500       0.0
536502       0.0
536508       0.0
536514       0.0
536520       0.0
5365

In [68]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1
basket_encoded = basket_France.applymap(hot_encode) 
basket_France = basket_encoded 
basket_France

Description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A563185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
